measuring time to load data to GPU

In [5]:
import zlib
import numpy as np
from datasets import load_dataset
import torch

In [8]:
def entropy_decoder(compressed_img,original_shape):
    decompressed = zlib.decompress(compressed_img)
    ẑ = np.frombuffer(decompressed, dtype=np.int8)
    ẑ = ẑ.reshape(original_shape)
    return ẑ
def batch_entropy_decode(compressed_batch,original_size):
    z = torch.cat([
        torch.tensor(
            entropy_decoder(compressed_batch[i], original_size)
        ) 
        for i in range(batch_size)
    ])
    return z

In [7]:
dataset = load_dataset("danjacobellis/imagenet_RDAE_batched_250k",split='train').with_format("torch")
N_classes=1001;

In [37]:
%%time
for (i_batch, batch) in enumerate(dataset):
    latent_size = batch['latent_size']
    batch_size = latent_size[0];
    original_size = latent_size.numpy().copy();
    original_size[0] = 1;
    
    compressed_batch = batch['compressed_batch']
    z = batch_entropy_decode(compressed_batch,original_size);
    z.to("cuda")
    
    label = batch['label'].to(torch.int64)
    y = torch.zeros((batch_size,N_classes,1,1),dtype=torch.int16)
    y.scatter_(1, label.view(-1, 1, 1, 1), 1)
    y.to("cuda")
    
    z.detach()
    y.detach()

CPU times: user 10min 34s, sys: 15.8 s, total: 10min 50s
Wall time: 2min 32s


In [40]:
z.shape

torch.Size([64, 128, 45, 63])

In [41]:
y.shape

torch.Size([64, 1001, 1, 1])

```
64 x  128   x ~128 x ~128
64 x  256   x ~64  x ~64
64 x  512   x ~32  x ~32
64 x  1024  x ~16  x ~16
64 x  2048  x ~8   x ~8
64 x  4096  x ~4   x ~4
64 x  8192  x ~2   x ~2
64 x 16384  x  1   x  1
64 x  1000  x  1   x  1
```